In [ ]:
!pip install tensorflow==2.4.3

     |████████████████████████████████| 394.5 MB 394 bytes/s 
     |████████████████████████████████| 2.9 MB 39.9 MB/s 
     |████████████████████████████████| 3.8 MB 42.8 MB/s 
     |████████████████████████████████| 462 kB 42.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
import numpy as np

In [ ]:
tf.__version__

'2.4.3'

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data
NUM_WORDS = 20000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
MAX_LENGTH = 300

x_train_2D = np.zeros([25000,MAX_LENGTH],dtype=np.int32)
for i in range(x_train.shape[0]):
    if len(x_train[i]) >= MAX_LENGTH:
        x_train_2D[i,:] = x_train[i][0:MAX_LENGTH]
    else:
        x_train_2D[i,0:len(x_train[i])] = x_train[i]
        x_train_2D[i,len(x_train[i]):] = NUM_WORDS + 1

x_test_2D = np.zeros([25000,MAX_LENGTH],dtype=np.int32)
for i in range(x_test.shape[0]):
    if len(x_test[i]) >= MAX_LENGTH:
        x_test_2D[i,:] = x_test[i][0:MAX_LENGTH]
    else:
        x_test_2D[i,0:len(x_test[i])] = x_test[i]
        x_test_2D[i,len(x_test[i]):] = NUM_WORDS + 1

In [ ]:
x_test_2D.shape

(25000, 300)

In [ ]:
whos

Variable     Type               Data/Info
-----------------------------------------
MAX_LENGTH   int                300
NUM_WORDS    int                20000
i            int                24999
imdb         TFModuleWrapper    <module 'tensorflow.pytho<...>tasets/imdb/__init__.py'>
np           module             <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
tf           TFModuleWrapper    <module 'tensorflow' from<...>/tensorflow/__init__.py'>
x_test       ndarray            25000: 25000 elems, type `object`, 200000 bytes (195.3125 kb)
x_test_2D    ndarray            25000x300: 7500000 elems, type `int32`, 30000000 bytes (28.6102294921875 Mb)
x_train      ndarray            25000: 25000 elems, type `object`, 200000 bytes (195.3125 kb)
x_train_2D   ndarray            25000x300: 7500000 elems, type `int32`, 30000000 bytes (28.6102294921875 Mb)
y_test       ndarray            25000: 25000 elems, type `int64`, 200000 bytes (195.3125 kb)
y_train      ndarray            25000: 25

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Input, Bidirectional, Conv1D, MaxPool1D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop

In [ ]:
input = Input((MAX_LENGTH,)) # 300
x = Embedding(NUM_WORDS+2, 400)(input) # remove this layer if use separate embedding such as fasttext/pythai etc. Feed 3D embedding directly to model
x = Conv1D(32,3,padding='same',activation='relu')(x)
x = MaxPool1D()(x) #(batch,150,32)
x = Bidirectional( LSTM(100, return_sequences=True) )(x) #3D
x = Bidirectional( LSTM(100, return_sequences=False) )(x) #2D
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(input, out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 400)          8000800   
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 32)           38432     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 150, 32)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 200)          106400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense (Dense)                (None, 128)               25728 

In [ ]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001, momentum=0.9),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
model.fit(x_train_2D, y_train, epochs=5)

Epoch 1/5
782/782 [==============================] - 178s 182ms/step - loss: 0.6457 - accuracy: 0.5963
Epoch 2/5
782/782 [==============================] - 143s 183ms/step - loss: 0.3484 - accuracy: 0.8486
Epoch 3/5
782/782 [==============================] - 143s 182ms/step - loss: 0.2938 - accuracy: 0.8837
Epoch 4/5
782/782 [==============================] - 143s 183ms/step - loss: 0.2588 - accuracy: 0.9029
Epoch 5/5
782/782 [==============================] - 143s 182ms/step - loss: 0.2346 - accuracy: 0.9136


In [ ]:
model.evaluate(x_test_2D,  y_test, verbose=2)

782/782 - 50s - loss: 0.3322 - accuracy: 0.8765


[0.3321583867073059, 0.876479983329773]